# Model leukemia development and determine posterior distributions with pyABC

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
#from sympy import *
import pandas as pds
#import pymc3 as pm
import pyabc as pyabc
import os
#from numba import jit
import scipy.stats as st
from scipy.special import kl_div
import tempfile
from sklearn.metrics import mean_log_error
import random

In [ ]:
import os
os.chdir("/omics/groups/OE0603/internal/jolly/Buettner_Rieger")

In [ ]:
# function to sample from mixture distribution
def sample_from_mixture_gamma(num_samples,alpha1,beta1,alpha2,beta2,mixing_prob):
    '''sample from a mixture gamma distribution
    Arguments:
    num_samples  -- sample size
    alpha1       -- shape parameter of the first gamma distr.
    beta1        -- rate parameters of the first gamma distr.
    alpha2       -- shape parameter of the sec. gamma distr.
    beta2        -- rate parameters of the sec. gamma distr.
    mixing_prob  -- probability to sample from the first gamma distr.
    '''
    # Generate random probabilities for each sample
    random_probs = np.random.random(num_samples)
    # Create a boolean mask for samples to be drawn from the first gamma distribution
    mask = random_probs < mixing_prob
    # Generate samples from the mixture gamma distribution
    samples = np.empty(num_samples)
    samples[mask] = np.random.gamma(alpha1, scale=beta1, size=mask.sum())
    samples[~mask] = np.random.gamma(alpha2, scale=beta2, size=(~mask).sum())
    return samples

In [ ]:
def calculate_y(params):
    '''analytical solution of the ODE model for one-way differentiation
    Arguments:
    p_1          -- proliferation rate of cluster of origin
    p_2          -- proliferation rate of cluster 2
    p_3          -- proliferation rate of cluster 3
    d_1_2        -- differentiation rate from cluster of origin to cluster 2
    d_1_3        -- differentiation rate from cluster of origin to cluster 3
    t            -- time (in days)
    n_1          -- initial number of cells in cluster of origin
    '''
    p_1, p_2,p_3,d_1_2,d_1_3,d_2_3,t, n_1 = params
    y_1 = n_1*np.exp(-t*(d_1_2 + d_1_3 - p_1))
    y_2 = d_1_2*n_1*np.exp(p_2*t)/(d_1_2 + d_1_3 - p_1 + p_2) - d_1_2*n_1*np.exp(-t*(d_1_2 + d_1_3 - p_1))/(d_1_2 + d_1_3 - p_1 + p_2)
    y_3 = d_1_3*n_1*np.exp(p_3*t)/(d_1_2 + d_1_3 - p_1 + p_3) - d_1_3*n_1*np.exp(-t*(d_1_2 + d_1_3 - p_1))/(d_1_2 + d_1_3 - p_1 + p_3)
    return y_1, y_2, y_3

In [ ]:
# Define parameters

# Define the P matrix (numerically substituted)

def calculate_y_back(params):
    '''ODE model for plasticity
            Arguments:
    p_1          -- proliferation rate of cluster of origin
    p_2          -- proliferation rate of cluster 2
    p_3          -- proliferation rate of cluster 3
    d_1_2        -- differentiation rate between cluster of origin and cluster 2
    d_1_3        -- differentiation rate between cluster of origin and cluster 3
    d_2_3        -- differentiation rate between cluster 2 and cluster 3    
    t            -- time (in days)
    n_1          -- initial number of cells in cluster of origin
    '''
    p_1, p_2,p_3,d_1_2,d_1_3,d_2_3 ,t, n_1= params[0:8]    
    def ode_system(t,y,p_1, p_2,p_3,d_1_2,d_1_3,d_2_3):
        dydt = [
        (p_1 -  d_1_2 + d_1_3) * y[0] + d_1_2 * y[1] + d_1_3 * y[2],  # dy1/dt
        d_1_2 * y[0] + (p_2 - d_1_2-d_2_3) * y[1]+d_2_3 * y[2],                  # dy2/dt
        d_1_3 * y[0] + (p_3 - d_1_3-d_2_3) * y[2]+d_2_3 * y[1]                # dy3/dt
        ]
        return dydt
    y0 = [n_1, 0, 0]  
    t_span = (0, t)  
# Solve the system
    solution = solve_ivp(ode_system, t_span, y0, t_eval=[t], method='RK45',args=(p_1, p_2,p_3,d_1_2,d_1_3,d_2_3))
    sol = solution.y
    result = tuple(np.float64(x) for x in sol.flatten())
    return result



In [ ]:
def model(paraMatrix):
    ''' simulates the one-way differentiation model for every clones, returns population size at for each clone and each cluster at t end
    '''
    iterable=tuple(paraMatrix)
    Simres=np.asanyarray(list(map(lambda params: calculate_y(params),iterable)))
    return Simres


In [ ]:
def model_backAndForth(paraMatrix):
    ''' simulates the plasticity model for every clones,, returns population size at for each clone and each cluster at t end
    '''
    iterable=tuple(paraMatrix)
    Simres=np.asanyarray(list(map(lambda params: calculate_y_back(params),iterable)))
    return Simres 

In [ ]:

num_clones = 200 # number of clones 
num_param= 6


T=50 # Final time of simulation, time of death in 
paraMatrix = np.zeros([num_clones,num_param+1])


parameters={"M_d_1_2_rate":0.166524,"alpha_d":553.402914,"mixing_prob_1_2":1.351263,"M_d_1_3_rate":0.689444,"mixing_prob_1_3":0.579466,"alpha_pr":84.913724,"M_pr1_rate":0.169292,"M_pr2_rate":0.37532,"M_pr3_rate":0.36426,"M_dlow_rate":0.000001,"mixing_prob_2_3":0.579466,"M_d_2_3_rate":0.579466}
def parameterMatrix(parameters,T):
    '''Generates model parameters for each clones sampled from the distributions with parameters fitted by the ABC framework
    '''
    paraMatrix = np.zeros([num_clones,num_param+2]) # matrix of dynamical parameters for each clone
    M_d_1_2_rate = parameters["M_d_1_2_rate"] # average "active" differentiation rate cluster 1  <-> or -> cluster 2
    alpha_d = parameters["alpha_d"] # shape parameter of the distributions of differentiation rates 
    beta_d_1_2 = M_d_1_2_rate/alpha_d  # scale parameter of the distribution of active differentiation rates cluster 1 <-> or -> cluster 2
    mixing_prob_1_2 = parameters["mixing_prob_1_2"] # Probability of sampling from the first gamma distribution for differentiation rates  1 <-> or -> 2
    M_d_1_3_rate= parameters["M_d_1_3_rate"]# average "active" differentiation rate cluster 1 <-> or ->  cluster 3
    beta_d_1_3 = M_d_1_3_rate/alpha_d   # scale parameter of the distribution for active differentiation rates cluster 1 <-> or -> cluster 3
    M_dlow_rate = parameters["M_dlow_rate"] # average low/background differentiation rate 
    beta_dlow = M_dlow_rate/alpha_d # scale parameter of the distribution for background differentiation rates
    mixing_prob_1_3 = parameters["mixing_prob_1_3"]  # Probability of sampling from the first gamma distribution for differentiation rates 1<-> or -> 3
    #shape parameter is common to every compartment proliferation
    alpha_pr = parameters["alpha_pr"] # shape parameter of the distributions of proliferation rates 
    M_pr1_rate = parameters["M_pr1_rate"] # average proliferation rate in cluster 1 
    beta_pr1 = M_pr1_rate/alpha_pr # scale parameter of the distribution for proliferation rates in cluster 1 
    M_pr2_rate = parameters["M_pr2_rate"] # scaling factor for the proliferation rate in cluster 2
    M_pr3_rate = parameters["M_pr3_rate"] # scaling factor for the proliferation rate in cluster 3
    p_r_1 = paraMatrix[:,0]
    p_r_2 = paraMatrix[:,1]
    p_r_3 = paraMatrix[:,2]
    d_r_1_2= paraMatrix[:,3]
    d_r_1_3 = paraMatrix[:,4]
    p_r_1[0:num_clones] = np.random.gamma(alpha_pr, beta_pr1, size=num_clones) #sampled proliferation rates for each clone in cluster 1
    p_r_2[0:num_clones] = p_r_1[0:num_clones]*M_pr2_rate # adjusted proliferation rates for cluster 2
    p_r_3[0:num_clones] = p_r_1[0:num_clones]*M_pr3_rate  # adjusted proliferation rates for cluster 3
    d_r_1_2[0:num_clones] = sample_from_mixture_gamma(num_clones,alpha_d,beta_dlow,alpha_d,beta_d_1_2,mixing_prob_1_2) #sampled differentiation rates for each clone cl 1 <-> or -> 2
    d_r_1_3[0:num_clones] = sample_from_mixture_gamma(num_clones,alpha_d,beta_dlow,alpha_d,beta_d_1_3,mixing_prob_1_3) #sampled differentiation rates for each clone cl 1 <-> or ->  3
    d_r_2_3 = paraMatrix[:,5]
    M_d_2_3_rate= parameters["M_d_2_3_rate"] # average "active" differentiation rate cluster 2  <->  cluster 3
    mixing_prob_2_3 = parameters["mixing_prob_2_3"] #probability of sampling from the first gamma distribution for differentiation rates  2 <-> 3
    beta_d_2_3 = M_d_2_3_rate/alpha_d # scale parameter of the distribution of active differentiation rate cluster 2  <->  cluster 3
    d_r_2_3 = sample_from_mixture_gamma(num_clones,alpha_d,beta_dlow,alpha_d,beta_d_2_3,mixing_prob_2_3) #sampled differentiation rates for each clone cl 2 <-> 3
    paraMatrix[:,6] = np.repeat(T,num_clones) # final time 
    #and now for the initial number of cells per clone in cluster 1 
    mean_n = 5
    alpha_n = 10
    beta_n = mean_n / alpha_n 
    paraMatrix[:,7] = np.round(np.random.gamma(alpha_n, beta_n, size=num_clones))
    return paraMatrix


In [ ]:
def get_ordered_outPut(Simres,DataSize,sample,OutputOrder):
    '''from the result of the model simulation, generate samples of the same size as the overall single-cell dataset, then 
    Argument:
    Simres       -- output of model() or modelbackandforth() functions
    DataSize     -- total number of single cell for the given mouse
    sample       -- proportion of non barcoded cells in the data per cluster 
    OutputOrder  -- 3 possible ordering of clusters with the source cluster positionned in 1st, 2nd or 3rd column
    '''
    flat_replications = Simres.flatten(order='F')
    flat_replications=flat_replications.astype("float64")
    flat_indices = np.arange(0,len(flat_replications),1)
    sampled_items = np.random.choice(flat_indices, size=DataSize, p=flat_replications/np.sum(flat_replications)) # sampling as many cells as there are in the datasets
    item_counts = np.bincount(sampled_items,minlength=num_clones*3)
    item_counts_clust1 = item_counts[0:num_clones]
    item_counts_clust2 = item_counts[num_clones:num_clones*2]
    item_counts_clust3 = item_counts[num_clones*2:num_clones*3]
    total_counts = item_counts_clust1+item_counts_clust2+item_counts_clust3
    def reduce_cluster(cluster, sample): #sampling the same proporition of labeled cells per cluster as in the data
        sample_size=np.sum(cluster)-np.int64(sample*np.sum(cluster))
        expanded_cluster = np.repeat(np.arange(len(cluster)), cluster)
        sampled_counts = np.random.choice(expanded_cluster, size=sample_size, replace=False)
        return np.bincount(sampled_counts, minlength=len(cluster))
    # ordering the output with source cluster in 1st, 2nd or 3rd column
    if OutputOrder==0:
        ordered_clust2red = reduce_cluster(item_counts_clust2, sample[0])
        ordered_clust1red = reduce_cluster(item_counts_clust1, sample[1])
        ordered_clust3red = reduce_cluster(item_counts_clust3, sample[2])
        total_counts= ordered_clust2red+ordered_clust1red+ordered_clust3red
        order=np.argsort(total_counts)[-len(total_counts):][::-1]
        ToReturn = np.vstack([ordered_clust2red[order], ordered_clust1red[order], ordered_clust3red[order]]).T        
    # Cluster 1, 2, 3
    elif OutputOrder==1:
        ordered_clust1red = reduce_cluster(item_counts_clust1, sample[0])
        ordered_clust2red = reduce_cluster(item_counts_clust2, sample[1])
        ordered_clust3red = reduce_cluster(item_counts_clust3, sample[2])
        total_counts= ordered_clust2red+ordered_clust1red+ordered_clust3red
        order=np.argsort(total_counts)[-len(total_counts):][::-1]
        ToReturn = np.vstack([ordered_clust1red[order], ordered_clust2red[order], ordered_clust3red[order]]).T
     # Cluster 2, 3, 1
    elif OutputOrder==2:
        ordered_clust2red = reduce_cluster(item_counts_clust2, sample[0])
        ordered_clust3red = reduce_cluster(item_counts_clust3, sample[1])
        ordered_clust1red = reduce_cluster(item_counts_clust1, sample[2])
        total_counts = ordered_clust2red+ordered_clust1red+ordered_clust3red
        order = np.argsort(total_counts)[-len(total_counts):][::-1]
        ToReturn = np.vstack([ordered_clust2red[order], ordered_clust3red[order], ordered_clust1red[order]]).T
    overall_sum = np.sum(ToReturn)
    # compute the relative size of each clone
    row_fractions = np.sum(ToReturn, axis=1) / overall_sum
    # For each clone, compute the distribution across clusters
    row_sums = np.sum(ToReturn, axis=1, keepdims=True)  # Row sums
    row_distributions = np.zeros_like(ToReturn, dtype=float)  # Initialize distribution matrix
    # Avoid division by zero for rows with sum = 0
    non_zero_rows = row_sums.squeeze() > 0 
    row_distributions[non_zero_rows] = ToReturn[non_zero_rows] / row_sums[non_zero_rows]   
    ToReturn = np.hstack((row_distributions,row_fractions[:, np.newaxis]))
    ToReturn = ToReturn.flatten(order="F")
    ToReturn = ToReturn.astype(float)
    return ToReturn


In [ ]:
def concatenate_outputs(returnedOutput1,returnedOutput2):
    '''concatenate outputs of get_ordered_outPut() for the 2 mice
    '''
    final_output = np.concatenate((returnedOutput1,returnedOutput2))
    return final_output

In [ ]:
# loading data into proper format

data_1=pds.read_csv("latestbarcodeDist72.csv")
data_2=pds.read_csv("latestbarcodeDist73.csv")
num_clones = 200

dataNP1=data_1.to_numpy()
sample_1=dataNP1[0,1:4]/np.sum(dataNP1[:,1:4],axis=0)
DataSize_1=np.sum(dataNP1[:,1:4])
dataNP1=dataNP1[1:,:]
orderdataNP1=np.argsort(np.sum(dataNP1[:,1:4],axis=1))[::-1]
np.arange(0,len(orderdataNP1),1)
backToOrder=[np.arange(0,len(orderdataNP1),1),orderdataNP1]
ordereddataNP1=dataNP1[orderdataNP1,:]

import tempfile

data1clean = ordereddataNP1[:,1:4]

data1clean=np.asarray(data1clean)

#we add as many clones as simulated+1, the unlabeled
diff=(num_clones)-np.shape(data1clean)[0]

data1clean=np.vstack((data1clean, np.zeros([diff, 3])))


### getting porportion of cells (relative size of each clone and distribution across clusters per clone)

overall_sum = np.sum(data1clean)
row_fractions = np.sum(data1clean, axis=1) / overall_sum

# Compute row sums and identify non-zero rows
row_sums = np.sum(data1clean, axis=1, keepdims=True) 
non_zero_rows = row_sums.squeeze() > 0  # Boolean mask for rows with non-zero sum

# Initialize row_distributions with zeros
row_distributions = np.zeros_like(data1clean, dtype=float)

# Only normalize rows with non-zero sums
row_distributions[non_zero_rows] = data1clean[non_zero_rows] / row_sums[non_zero_rows]

data1clean=np.hstack((row_distributions,row_fractions[:, np.newaxis]))
data1clean=data1clean.flatten(order='F')
data1clean=data1clean.astype(float)

################################
########same as above for mouse 2
################################

dataNP2=data_2.to_numpy()
sample_2=dataNP2[0,1:4]/np.sum(dataNP2[:,1:4],axis=0)
DataSize_2=np.sum(dataNP2[:,1:4])
dataNP2=dataNP2[1:,:]
orderdataNP2=np.argsort(np.sum(dataNP2[:,1:4],axis=1))[::-1]
np.arange(0,len(orderdataNP2),1)
backToOrder=[np.arange(0,len(orderdataNP2),1),orderdataNP2]
ordereddataNP2=dataNP2[orderdataNP2,:]
#ordereddataNP2[:,1:4]=ordereddataNP2[:,1:4]/np.sum(ordereddataNP2[:,1:4],axis=0)

import tempfile

data2clean = ordereddataNP2[:,1:4]
#abc.new("sqlite:///mouse73CloneSizeDist.db" , {"data": data2clean})
data2clean=np.asarray(data2clean)
#we add as many clones as simulated+1, the unlabeled
diff=(num_clones)-np.shape(data2clean)[0]
#print(data2clean)
#print(np.zeros([42,4]))
#print(ordereddataNP2[:,1:4])
data2clean=np.vstack((data2clean, np.zeros([diff, 3])))
#print(data2clean)
#print(DataSize)
#print(ordereddataNP2)
overall_sum = np.sum(data2clean)
row_fractions = np.sum(data2clean, axis=1) / overall_sum

# Compute row sums and identify non-zero rows
row_sums = np.sum(data2clean, axis=1, keepdims=True)  # Keep dimensions for broadcasting
non_zero_rows = row_sums.squeeze() > 0  # Boolean mask for rows with non-zero sum

# Initialize row_distributions with zeros
row_distributions = np.zeros_like(data2clean, dtype=float)

# Only normalize rows with non-zero sums
row_distributions[non_zero_rows] = data2clean[non_zero_rows] / row_sums[non_zero_rows]

data2clean=np.hstack((row_distributions,row_fractions[:, np.newaxis]))


data2clean=data2clean.flatten(order='F')
data2clean=data2clean.astype(float)


# concatenate the 2 mice
sample=list([sample_1,sample_2])


observation=np.concatenate((data1clean,data2clean))

DataSize=list([DataSize_1,DataSize_2])






In [ ]:
class bigModels:
    '''generates wrapper functions which produce final outputs to be compared to data from a parameter set
    Argument:
    DataSize     -- list with 2 elements: total number of single cell for each of the mice
    T            -- T end for the simulations
    sample       -- proportion of non barcoded cells in the data per cluster (column) and per mouse (row)
    OutputOrder  -- 3 possible ordering of clusters with the source cluster positionned in 1st, 2nd or 3rd column
    '''    
    def __init__(self,DataSize,T,sample,OutputOrder):
          self.DataSize, self.T,self.sample,self.OutputOrder = DataSize,T,sample,OutputOrder
    def __call__(self, parameters):
        paraMatrix_1=parameterMatrix(parameters,self.T)
        paraMatrix_2=parameterMatrix(parameters,self.T)
        Simres_1=model(paraMatrix_1)
        Simres_2=model(paraMatrix_2)
        #print(Simres)
        ordered_outPut1 = get_ordered_outPut(Simres_1,self.DataSize[0],self.sample[0],self.OutputOrder)
        ordered_outPut2 = get_ordered_outPut(Simres_2,self.DataSize[1],self.sample[1],self.OutputOrder)
        ordered_output = concatenate_outputs(ordered_outPut1,ordered_outPut2)
        return {"data": ordered_output}

In [ ]:
class bigModels_back:
     '''same as bigModels for plasticity models
    '''    
    def __init__(self,DataSize,T,sample,OutputOrder):
          self.DataSize, self.T,self.sample,self.OutputOrder = DataSize,T,sample,OutputOrder
    def __call__(self, parameters):
        paraMatrix_1=parameterMatrix(parameters,self.T)
        paraMatrix_2=parameterMatrix(parameters,self.T)
        Simres_1=model_backAndForth(paraMatrix_1)
        Simres_2=model_backAndForth(paraMatrix_2)
        #print(Simres)
        ordered_outPut1 = get_ordered_outPut(Simres_1,self.DataSize[0],self.sample[0],self.OutputOrder)
        ordered_outPut2 = get_ordered_outPut(Simres_2,self.DataSize[1],self.sample[1],self.OutputOrder)
        ordered_output =concatenate_outputs(ordered_outPut1,ordered_outPut2)
        return {"data": ordered_output}

In [ ]:

def mean_variance_per_block(data, block_size=6, num_blocks=7):
    """
    Compute mean and variance per column for blocks of rows.

    Parameters:
    - data: np.array of shape (>=30, 3) where we compute stats on the first 30 rows.
    - block_size: Size of each block (default=10).
    - num_blocks: Number of blocks to compute (default=3).

    Returns:
    - means: np.array of shape (num_blocks, 3) with mean per column per block.
    - variances: np.array of shape (num_blocks, 3) with variance per column per block.
    """
    means = []
    variances = []

    for i in range(num_blocks):
        block = data[i * block_size : (i + 1) * block_size]  # Extract block
        means.append(np.mean(block, axis=0))  # Mean per column
        variances.append(np.std(block, axis=0))  # Variance per column

    return np.array(means), np.array(variances)



In [ ]:
def distance(sim_data,obs_data, block_size=6, num_blocks=7):
    """
    Compute distance between observed and simulated block means and variances.
    Parameters:
    - obs_data: np.array of shape (>=30, 3) - first 30 rows of observed data.
    - sim_data: np.array of shape (>=30, 3) - first 30 rows of simulated data.
    - block_size: Number of rows per block (default=10).
    - num_blocks: Number of blocks (default=3).
    Returns:
    - total_distances: np.array of shape (num_blocks,), total  distance per block.
    """ 
    obs_data_1 = obs_data["data"][0:num_clones*4]
    sim_data_1 = sim_data["data"][0:num_clones*4]
    obs_data_2 = obs_data["data"][num_clones*4:]
    sim_data_2 = sim_data["data"][num_clones*4:]
    shapes=np.shape(obs_data_1)

    # compute for first mouse
    obs_data_1 = obs_data_1.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_1 = obs_data_1[:, [0, 1,2, 3]]
    sim_data_1 = np.reshape(sim_data_1,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_1 = sim_data_1[:, [0, 1,2, 3]]
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_1, obs_variances_1 = mean_variance_per_block(obs_data_1, block_size, num_blocks)
    sim_means_1, sim_variances_1 = mean_variance_per_block(sim_data_1, block_size, num_blocks)

    # Compute  Euclidean distance for means
    distance_means_1 = np.sum(np.abs(obs_means_1 - sim_means_1), axis=1)

    # Compute  Euclidean distance for variances
    distance_variances_1 = np.sum(np.abs(obs_variances_1 - sim_variances_1), axis=1)

    # Total  distance = sum of both
    total_distances_1 = distance_means_1 + distance_variances_1

    # compute for second mouse
    
    obs_data_2 = obs_data_2.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_2 = obs_data_2[:, [0, 1,2, 3]]
    sim_data_2 = np.reshape(sim_data_2,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_2 = sim_data_2[:, [0, 1,2, 3]]
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_2, obs_variances_2 = mean_variance_per_block(obs_data_2, block_size, num_blocks)
    sim_means_2, sim_variances_2 = mean_variance_per_block(sim_data_2, block_size, num_blocks)

    # Compute  Euclidean distance for means
    distance_means_2 = np.sum(np.abs(obs_means_2 - sim_means_2), axis=1)
    # Compute  Euclidean distance for variances
    distance_variances_2 = np.sum(np.abs(obs_variances_2 - sim_variances_2), axis=1)

    # Total  distance = sum of both
    total_distances = distance_means_1 + distance_variances_1 + distance_means_2 + distance_variances_2
    return np.sum(total_distances)

In [ ]:
# functions to simulate each of the 6 models
bigModel_1=bigModels(DataSize,T,sample,0)
bigModel_2=bigModels(DataSize,T,sample,1)
bigModel_3=bigModels(DataSize,T,sample,2)
bigModel_4=bigModels_back(DataSize,T,sample,0)
bigModel_5=bigModels_back(DataSize,T,sample,1)
bigModel_6=bigModels_back(DataSize,T,sample,2)


models = [bigModel_1,bigModel_2,bigModel_3,bigModel_4,bigModel_5,bigModel_6]

# prior distribution, shared by all 6 models
prior=pyabc.Distribution(M_d_1_2_rate=pyabc.RV("uniform", 0.1, 0.6),alpha_d=pyabc.RV("uniform", 200, 600),mixing_prob_1_2=pyabc.RV("uniform", 0.5, 0.4),
M_d_1_3_rate=pyabc.RV("uniform", 0.1, 0.6),mixing_prob_1_3=pyabc.RV("uniform", 0.5, 0.4),
alpha_pr=pyabc.RV("uniform", 5, 1000),M_pr1_rate=pyabc.RV("uniform", 0.1, 0.6),M_pr2_rate=pyabc.RV("norm", 1, 0.3),M_pr3_rate=pyabc.RV("norm", 1, 0.3),M_dlow_rate=pyabc.RV("uniform", 0.000001, 0.000001),mixing_prob_2_3=pyabc.RV("uniform", 0.5, 0.5),M_d_2_3_rate=pyabc.RV("uniform", 0.1, 0.6))


priors = [prior,prior,prior,prior,prior,prior]


# prepare sampler
abc = pyabc.ABCSMC(models, priors, distance, population_size=8000,sampler=pyabc.MulticoreEvalParallelSampler(n_procs=8))#,sampler =pyabc.SingleCoreSampler())


db_path = os.path.join(tempfile.gettempdir(), "sim.db")
history = abc.new("sqlite:///" + db_path, {"data": observation})





In [ ]:
#run sampler
abc.run(min_acceptance_rate=0.01)

In [ ]:
model_probabilities = history.get_model_probabilities()
model_probabilities

# Generate new model simulations from the posterior distribution

In [ ]:

#prepare posterior distribution for model 4 to use as input for model simulation

model_3_data = df_pop[df_pop['m'] == 3]

# Now, get the 'sumstat_val' for those rows and group every 10 rows into an array
sumstat_values_model_3 = []

# Iterate over the rows in chunks of 12
for i in range(0, len(model_3_data), 12):
    # Extract the sumstat_val for the current set of 12 rows
    sumstat_chunk = model_3_data['sumstat_val'].to_numpy()[i]
    sumstat_values_model_3.append(sumstat_chunk)

par_names_model_3 = []

for i in range(0, len(model_3_data), 12):
    # Extract the sumstat_val for the current set of 10 rows
    parnames_chunk = model_3_data['par_name'].to_numpy()[i:i+12]
    par_names_model_3.append(parnames_chunk)

par_values_model_3 = []

for i in range(0, len(model_3_data), 12):
    # Extract the sumstat_val for the current set of 12 rows
    par_chunk = model_3_data['par_val'].to_numpy()[i:i+12]
    par_values_model_3.append(par_chunk)

dictPred_3=[]

for i in range(0, len(par_names_model_3)):
    dictPred_3.append(dict(zip(par_names_model_3[i],par_values_model_3[i])))



In [ ]:
#we prepare posterior distribution for model 1

model_0_data = df_pop[df_pop['m'] == 0]

# Now, get the 'sumstat_val' for those rows and group every 10 rows into an array
sumstat_values_model_0 = []

# Iterate over the rows in chunks of 12
for i in range(0, len(model_0_data), 12):
    # Extract the sumstat_val for the current set of 12 rows
    sumstat_chunk = model_0_data['sumstat_val'].to_numpy()[i]
    sumstat_values_model_0.append(sumstat_chunk)

par_names_model_0 = []

for i in range(0, len(model_0_data), 12):
    # Extract the sumstat_val for the current set of 10 rows
    parnames_chunk = model_0_data['par_name'].to_numpy()[i:i+12]
    par_names_model_0.append(parnames_chunk)

par_values_model_0 = []

for i in range(0, len(model_0_data), 12):
    # Extract the sumstat_val for the current set of 12 rows
    par_chunk = model_0_data['par_val'].to_numpy()[i:i+12]
    par_values_model_0.append(par_chunk)

dictPred_0=[]

for i in range(0, len(par_names_model_0)):
    dictPred_0.append(dict(zip(par_names_model_0[i],par_values_model_0[i])))


In [ ]:
All_pop=history.get_all_populations()
All_pop.to_csv("ModelfitPopCombinedCellPlasticity_200Clones_samp_PopStat.csv")
model_probabilities = history.get_model_probabilities()
model_probabilities.to_csv("ModelfitPopCombinedCellPlasticity_200Clones_samp_proba.csv")
model_probabilitiesnump=model_probabilities.to_numpy()
shape=np.shape(model_probabilitiesnump)[0]
print(shape)
df_pop=history.get_population_extended(t=shape-2)

df_pop.to_csv("ModelfitPopCombinedCellPlasticity_200Clonessamp_posterior.csv")

## return distances for each value of the summary statistics separately

In [ ]:
# same as distance() except distance_2 does not sum the distances

def distance_2(sim_data,obs_data, block_size=6, num_blocks=7):
    """
    Compute  distance between observed and simulated block means and variances.
    Parameters:
    - obs_data: np.array of shape (>=30, 3) - first 30 rows of observed data.
    - sim_data: np.array of shape (>=30, 3) - first 30 rows of simulated data.
    - block_size: Number of rows per block (default=10).
    - num_blocks: Number of blocks (default=3).
    Returns:
    - total_distances: np.array of shape (num_blocks,), total  distance per block.
    """ 
    obs_data_1 = obs_data["data"][0:num_clones*4]
    sim_data_1 = sim_data["data"][0:num_clones*4]
    obs_data_2 = obs_data["data"][num_clones*4:]
    sim_data_2 = sim_data["data"][num_clones*4:]
    shapes=np.shape(obs_data_1)

    # compute for first mouse
    obs_data_1 = obs_data_1.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_1 = obs_data_1
    sim_data_1 = np.reshape(sim_data_1,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_1 = sim_data_1
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_1, obs_variances_1 = mean_variance_per_block(obs_data_1, block_size, num_blocks)
    sim_means_1, sim_variances_1 = mean_variance_per_block(sim_data_1, block_size, num_blocks)

    # Compute  Euclidean distance for means
    distance_means_1 = np.abs(obs_means_1 - sim_means_1)

    # Compute  Euclidean distance for variances
    distance_variances_1 =np.abs(obs_variances_1 - sim_variances_1)

    # Total distance = sum of both
    total_distances_1 = distance_means_1 + distance_variances_1

    # compute for second mouse
    
    obs_data_2 = obs_data_2.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_2 = obs_data_2
    sim_data_2 = np.reshape(sim_data_2,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_2 = sim_data_2
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_2, obs_variances_2 = mean_variance_per_block(obs_data_2, block_size, num_blocks)
    sim_means_2, sim_variances_2 = mean_variance_per_block(sim_data_2, block_size, num_blocks)

    # Compute Euclidean distance for means
     distance_means_2 = np.abs(obs_means_2 - sim_means_2)
    # Compute Euclidean distance for variances
    distance_variances_2 = np.abs(obs_variances_2 - sim_variances_2)

    distances = np.vstack((distance_means_1,distance_variances_1,distance_means_2,distance_variances_2))
   
    return distances

In [ ]:
#distance_3 same as distance_2 but with relative distance instead of absolute

def distance_3(sim_data,obs_data, block_size=6, num_blocks=7):
    """
    Compute distance between observed and simulated block means and variances.
    Parameters:
    - obs_data: np.array of shape (>=30, 3) - first 30 rows of observed data.
    - sim_data: np.array of shape (>=30, 3) - first 30 rows of simulated data.
    - block_size: Number of rows per block (default=10).
    - num_blocks: Number of blocks (default=3).
    Returns:
    - total_distances: np.array of shape (num_blocks,), total distance per block.
    """ 
    obs_data_1 = obs_data["data"][0:num_clones*4]
    sim_data_1 = sim_data["data"][0:num_clones*4]
    obs_data_2 = obs_data["data"][num_clones*4:]
    sim_data_2 = sim_data["data"][num_clones*4:]
    shapes=np.shape(obs_data_1)

    # compute for first mouse
    obs_data_1 = obs_data_1.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_1 = obs_data_1
    sim_data_1 = np.reshape(sim_data_1,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_1 = sim_data_1
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_1, obs_variances_1 = mean_variance_per_block(obs_data_1, block_size, num_blocks)
    sim_means_1, sim_variances_1 = mean_variance_per_block(sim_data_1, block_size, num_blocks)

    # Compute Euclidean distance for means
    distance_means_1 = obs_means_1 - sim_means_1

    # Compute Euclidean distance for variances
    distance_variances_1 =obs_variances_1 - sim_variances_1

    # Total distance = sum of both
    total_distances_1 = distance_means_1 + distance_variances_1

    # compute for second mouse
    
    obs_data_2 = obs_data_2.reshape((np.int64(shapes[0]/4),4),order="F")
    obs_data_2 = obs_data_2
    sim_data_2 = np.reshape(sim_data_2,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_2 = sim_data_2
    #obs_data=obs_data[:,0:4]
    #sim_data=sim_data[:,0:4]
    # Compute block-wise means and variances for observed and simulated data
    obs_means_2, obs_variances_2 = mean_variance_per_block(obs_data_2, block_size, num_blocks)
    sim_means_2, sim_variances_2 = mean_variance_per_block(sim_data_2, block_size, num_blocks)

    # Compute Euclidean distance for means
    distance_means_2 = obs_means_2 - sim_means_2
    # Compute Euclidean distance for variances
    distance_variances_2 = obs_variances_2 - sim_variances_2

    distances = np.vstack((distance_means_1,distance_variances_1,distance_means_2,distance_variances_2))

    return distances

## generate 1000 simulations from posterior parameter distribution for model 1 and return distances 

In [14]:
n = 1000  
num_clones=200
sampled_indices = random.sample(range(len(dictPred_0)), n)
Pred_0 = []
distPred_0 = []
dist2Pred_0 = []
dist3Pred_0 = []
for i in sampled_indices:
    sim_data=bigModel_1(dictPred_0[i])
    sim_data_1 = sim_data["data"][0:num_clones*4]
    sim_data_2 = sim_data["data"][num_clones*4:]
    shapes=np.shape(sim_data_1)
    sim_data_1 = np.reshape(sim_data_1,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_2 = np.reshape(sim_data_2,shape=(np.int64(shapes[0]/4),4),order="F")
    predReorg=np.vstack((sim_data_1,sim_data_2))
    Pred_0.append(predReorg) 
    distPred_0.append(distance(bigModel_1(dictPred_0[i]), {"data": observation}))
    dist2Pred_0.append(distance_2(bigModel_1(dictPred_0[i]),{"data":observation}))
    dist3Pred_0.append(distance_3(bigModel_1(dictPred_0[i]),{"data":observation}))

NameError: name 'dictPred_0' is not defined

In [ ]:
#we generate 1000 simulations from the posterior distribution, here for model 3
Pred_3 = []
distPred_3 = []
dist2Pred_3 = []
dist3Pred_3 = []
n = 1000  
num_clones=200
sampled_indices = np.random.choice(range(len(dictPred_3)), n)
Pred_3 = []
for i in sampled_indices:
    sim_data=bigModel_4(dictPred_3[i])
    sim_data_1 = sim_data["data"][0:num_clones*4]
    sim_data_2 = sim_data["data"][num_clones*4:]
    shapes=np.shape(sim_data_1)
    sim_data_1 = np.reshape(sim_data_1,shape=(np.int64(shapes[0]/4),4),order="F")
    sim_data_2 = np.reshape(sim_data_2,shape=(np.int64(shapes[0]/4),4),order="F")
    predReorg=np.vstack((sim_data_1,sim_data_2))
    Pred_3.append(predReorg) 
    distPred_3.append(distance(bigModel_4(dictPred_3[i]), {"data": observation}))
    dist2Pred_3.append(distance_2(bigModel_4(dictPred_3[i]),{"data":observation}))
    dist3Pred_3.append(distance_3(bigModel_4(dictPred_3[i]),{"data":observation}))

In [ ]:
Pred_0=np.reshape(Pred_0,shape=(400000,4))
Pred_3=np.reshape(Pred_3,shape=(400000,4))
np.savetxt("predCRModel0.csv", Pred_0, delimiter=",")

np.savetxt("predCRModel3.csv", Pred_3, delimiter=",")



dist3Pred_0=np.reshape(dist3Pred_0,shape=(28000,4))
dist3Pred_3=np.reshape(dist3Pred_3,shape=(28000,4))
np.savetxt("dist3CRModel0.csv", dist3Pred_0, delimiter=",")
np.savetxt("dist3CRModel3.csv", dist3Pred_3, delimiter=",")